# Uppgift 2 - Sportstatistik
- Välj 2-4 sporter och skapa lämpliga grafer/diagram för att visualisera exempelvis:
- medaljfördelning mellan länder i sporterna, åldersfördelning i sporterna
- Skapa fler plots för att visualisera olika aspekter kring sporterna.

### Våra frågeställningar:
- Kolla igenom vilka sporter som varit med?
- Analysera "Tug-Of-War"! (dragkamp) Kul att tex kolla hur länge det varit med i OS, vilka har varit bäst? Osv.
- Finns det andra gamla eller intressanta sporter att titta på?
- Alpinism?
- Aeronautics [https://www.topendsports.com/events/demonstration/aeronautics.htm](https://www.topendsports.com/events/demonstration/aeronautics.htm)
- Yngsta, äldsta medaljörerna?
#### "Tug of war" (Dragkamp) (Daniel):
- Vilka länder har deltagit och hur ser medaljfördelningen ut?
- Vilka år var detta en sport i OS?

#### Kulstötning / Slägga / Spjut / Diskus (Daniel):
- Hur ser längd- och vikt-utvecklingen ut genom historien?
- Analysera män och kvinnor separat
- Hade varit intressant att jämföra med resultaten i sporterna också....
- Distribution plots? Box-plots? Eller en animation frame, som går igenom åren?

In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, callback, Output, Input
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
from dash import dash_table
from io import BytesIO
import base64
athlete_events = pd.read_csv("./Data/athlete_events.csv")

### 10 Bästa sporter för Frankrike

Här skapar jag en DataFrame med lämplig information för att plotta alla olika grafer som ska undersökas.

Informationen är:
- Totala medaljer vunnet av de top 10 sporterna.
- Av det totala, hur många av dem guld, silver och brons.
- Av varje guld silver och brons. Hur många var kvinnor och hur många var män.

In [76]:
only_france_df = athlete_events[athlete_events['Team'] == 'France']
only_france_df.head()
medal_counts = only_france_df.groupby('Sport')['Medal'].value_counts().unstack().fillna(0)
medal_counts[['Gold', 'Silver', 'Bronze']]
medal_counts['Total'] = medal_counts[['Gold', 'Silver', 'Bronze']].sum(axis=1)
medal_counts[['Gold', 'Silver', 'Bronze', 'Total']]
medal_counts['Participations'] = medal_counts.sum(axis=1)
medal_counts[['Gold', 'Silver', 'Bronze', 'Total', 'Participations']]
gold_female_counts = only_france_df[only_france_df['Medal'] == 'Gold'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Gold Female'] = gold_female_counts.fillna(0)
silver_female_counts = only_france_df[only_france_df['Medal'] == 'Silver'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Silver Female'] = silver_female_counts.fillna(0)

bronze_female_counts = only_france_df[only_france_df['Medal'] == 'Bronze'].loc[only_france_df['Sex'] == 'F'].groupby('Sport').size()
medal_counts['Bronze Female'] = bronze_female_counts.fillna(0)

gold_male_counts = only_france_df[only_france_df['Medal'] == 'Gold'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Gold Male'] = gold_male_counts.fillna(0)

silver_male_counts = only_france_df[only_france_df['Medal'] == 'Silver'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Silver Male'] = silver_male_counts.fillna(0)

bronze_male_counts = only_france_df[only_france_df['Medal'] == 'Bronze'].loc[only_france_df['Sex'] == 'M'].groupby('Sport').size()
medal_counts['Bronze Male'] = bronze_male_counts.fillna(0)

medal_counts['Total Female'] = medal_counts[['Gold Female', 'Silver Female', 'Bronze Female']].sum(axis=1)
medal_counts['Total Male'] = medal_counts[['Gold Male', 'Silver Male', 'Bronze Male']].sum(axis=1)
medal_counts.fillna(0, inplace=True)

medal_counts.head(10)

Medal,Bronze,Gold,Silver,Total,Participations,Gold Female,Silver Female,Bronze Female,Gold Male,Silver Male,Bronze Male,Total Female,Total Male
Sport,,,,,,,,,,,,,
Alpine Skiing,15.0,15.0,15.0,45.0,90.0,4.0,8.0,7.0,11.0,7.0,8.0,19.0,26.0
Archery,16.0,7.0,25.0,48.0,96.0,0.0,0.0,3.0,7.0,25.0,13.0,3.0,45.0
Art Competitions,5.0,6.0,4.0,15.0,30.0,0.0,1.0,0.0,6.0,3.0,5.0,1.0,14.0
Athletics,62.0,14.0,34.0,110.0,220.0,6.0,2.0,8.0,8.0,32.0,54.0,16.0,94.0
Basketball,0.0,0.0,38.0,38.0,76.0,0.0,12.0,0.0,0.0,26.0,0.0,12.0,26.0
Biathlon,24.0,8.0,8.0,40.0,80.0,4.0,5.0,10.0,4.0,3.0,14.0,19.0,21.0
Bobsleigh,4.0,0.0,0.0,4.0,8.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0
Boxing,10.0,6.0,9.0,25.0,50.0,1.0,1.0,0.0,5.0,8.0,10.0,2.0,23.0
Canoeing,29.0,8.0,12.0,49.0,98.0,1.0,1.0,4.0,7.0,11.0,25.0,6.0,43.0


In [77]:
top_10_sports_total = medal_counts.nlargest(10, 'Total')
top_10_sports_total_plot = px.bar(top_10_sports_total, x=top_10_sports_total.index, y=['Bronze', 'Silver', 'Gold'], title='Top 10 sports with the highest total medals won by France', barmode='stack')

top_10_sports_total_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Total medals won",
)
top_10_sports_total_plot.update_traces(marker_color='#CD7F32', selector=dict(name='Bronze'))
top_10_sports_total_plot.update_traces(marker_color='silver', selector=dict(name='Silver'))
top_10_sports_total_plot.update_traces(marker_color='gold', selector=dict(name='Gold'))
top_10_sports_total_plot.show()

In [81]:
top_10_sports_gold = medal_counts.nlargest(10, 'Gold')
top_10_sports_gold_plot = px.bar(top_10_sports_gold, x=top_10_sports_gold.index, y=['Gold Female', 'Gold Male'], title='Top 10 sports with the highest number of gold medals won by France', barmode='stack')
top_10_sports_gold_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of gold medals won"
)
top_10_sports_gold_plot.show()

In [82]:
top_10_sports_silver = medal_counts.nlargest(10, 'Silver')
top_10_sports_silver_plot = px.bar(top_10_sports_silver, x=top_10_sports_silver.index, y=['Silver Female', 'Silver Male'], title='Top 10 sports with the highest number of silver medals won by France', barmode='stack')
top_10_sports_silver_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of silver medals won"
)
top_10_sports_silver_plot.show()

In [83]:
top_10_sports_bronze = medal_counts.nlargest(10, 'Bronze')
top_10_sports_bronze_plot = px.bar(top_10_sports_bronze, x=top_10_sports_bronze.index, y=['Bronze Female', 'Bronze Male'], title='Top 10 sports with the highest number of bronze medals won by France', barmode='stack')
top_10_sports_bronze_plot.update_layout(
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_title="Number of bronze medals won"
)
top_10_sports_bronze_plot.show()

### Histogram över olika åldrar över tid

In [123]:
# Filter the dataframe to include only rows where the Team is 'France'
only_france_df = athlete_events[athlete_events['Team'] == 'France']
only_france_df.sort_values(by=['Year'], inplace=True)
only_france_df.dropna(subset=['Age'], inplace=True)

# Create a histogram of ages over time
histogram_ages_over_time = px.histogram(only_france_df, x='Age', title='Histogram of Ages Over Time', animation_frame='Year', range_x=[10, 80])

# Customize the layout of the histogram
histogram_ages_over_time.update_layout(
    xaxis_title='Age',
    yaxis_title='Count',
    paper_bgcolor="black",
    plot_bgcolor="black",
    font_color="white",
    yaxis_range=[0, 100]
)

# Show the histogram
histogram_ages_over_time.show()

/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_53225/2360890153.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_53225/2360890153.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [98]:
fencing_df = only_france_df[only_france_df['Sport'] == 'Fencing']



,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Age_Range
1202,676,Gaston Achille,M,NaN,NaN,NaN,France,FRA,1900 Summer,1900,Summer,Paris,Fencing,"Fencing Men's epee, Individual",NaN,nan-nan
1340,757,Adam,M,NaN,NaN,NaN,France,FRA,1900 Summer,1900,Summer,Paris,Fencing,"Fencing Men's epee, Individual",NaN,nan-nan
4467,2550,Antoine Luc Alessandri,M,57.0,NaN,NaN,France,FRA,1900 Summer,1900,Summer,Paris,Fencing,"Fencing Men's Sabre, Masters, Individual",NaN,50.0-59.0
4766,2723,Gaston Jules Louis Antoine Alibert,M,22.0,NaN,NaN,France,FRA,1900 Summer,1900,Summer,Paris,Fencing,"Fencing Men's epee, Individual",NaN,20.0-29.0
4767,2723,Gaston Jules Louis Antoine Alibert,M,30.0,NaN,NaN,France,FRA,1908 Summer,1908,Summer,London,Fencing,"Fencing Men's epee, Individual",Gold,30.0-39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26243,13681,Marcelle Bory,F,NaN,NaN,NaN,France,FRA,1924 Summer,1924,Summer,Paris,Fencing,"Fencing Women's Foil, Individual",NaN,nan-nan
26372,13726,Bruno Boscherie,M,29.0,180.0,74.0,France,FRA,1980 Summer,1980,Summer,Moskva,Fencing,"Fencing Men's Foil, Team",Gold,20.0-29.0
26720,13903,mile Bouard,M,NaN,NaN,NaN,France,FRA,1900 Summer,1900,Summer,Paris,Fencing,"Fencing Men's Foil, Masters, Individual",NaN,nan-nan
26868,13970,Saoussen Boudiaf,F,22.0,173.0,69.0,France,FRA,2016 Summer,2016,Summer,Rio de Janeiro,Fencing,"Fencing Women's Sabre, Team",NaN,20.0-29.0


# Weightlifting

In [39]:
# Create DataFrame with only sport = weightlifting
weightlifting = athlete_events.query("Sport == 'Weightlifting'") # DF med all weightlifting

#remove NaN's from weight, height and age:
weightlifting = weightlifting.dropna(subset='Weight')
weightlifting = weightlifting.dropna(subset='Height')
weightlifting = weightlifting.dropna(subset='Age')

# removes "weightlifting" from Event column:
weightlifting['Event'] = weightlifting['Event'].str.replace("Weightlifting ","")

display(weightlifting['Event'].unique()) # kollar vilka event som finns
px.box(weightlifting.query('''Event == "Women's Super-Heavyweight"'''), x='Year', y='Weight')
#display(type(weightlifting))

array(["Women's Super-Heavyweight", "Men's Lightweight",
       "Men's Middleweight", "Women's Featherweight", "Men's Heavyweight",
       "Men's Flyweight", "Men's Light-Heavyweight",
       "Men's Middle-Heavyweight", "Women's Lightweight",
       "Women's Light-Heavyweight", "Women's Heavyweight",
       "Men's Featherweight", "Women's Middleweight", "Women's Flyweight",
       "Men's Super-Heavyweight", "Men's Bantamweight",
       "Men's Heavyweight I", "Men's Heavyweight II",
       "Men's Unlimited, One Hand", "Men's Unlimited, Two Hands",
       "Men's All-Around Dumbbell Contest"], dtype=object)

# Speed skating

In [40]:
x = athlete_events['Sport'].str.contains('Speed Skating')
speedskating = athlete_events.loc[x]
speedskating['Event'].unique()

array(["Speed Skating Women's 500 metres",
       "Speed Skating Women's 1,000 metres",
       "Speed Skating Men's 500 metres",
       "Speed Skating Men's 1,500 metres",
       "Speed Skating Men's 5,000 metres",
       "Speed Skating Men's 10,000 metres",
       "Speed Skating Women's 1,500 metres",
       "Speed Skating Women's Team Pursuit (6 laps)",
       "Speed Skating Men's 1,000 metres",
       "Short Track Speed Skating Men's 5,000 metres Relay",
       "Speed Skating Women's 3,000 metres",
       "Short Track Speed Skating Women's 3,000 metres Relay",
       "Short Track Speed Skating Men's 500 metres",
       "Short Track Speed Skating Men's 1,000 metres",
       "Short Track Speed Skating Men's 1,500 metres",
       "Speed Skating Men's Team Pursuit (8 laps)",
       "Speed Skating Women's 5,000 metres",
       "Short Track Speed Skating Women's 500 metres",
       "Short Track Speed Skating Women's 1,000 metres",
       "Short Track Speed Skating Women's 1,500 metres",
 

In [41]:
# filtrerar dataframen så att bara de events vi är intresserade av kommer med:
speed_skating = athlete_events[
    (athlete_events['Event'] == "Speed Skating Men's 1,500 metres") |
    (athlete_events['Event'] == "Speed Skating Men's 500 metres") |
    (athlete_events['Event'] == "Speed Skating Men's 5,000 metres") |
    (athlete_events['Event'] == "Speed Skating Men's 10,000 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 1,000 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 1,500 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 500 metres") |
    (athlete_events['Event'] == "Speed Skating Men's 1,000 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 3,000 metres") |
    (athlete_events['Event'] == "Speed Skating Women's 5,000 metres")
    ]
speed_skating

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
5,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
6,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,NaN
7,5,Christine Jacoba Aaftink,F,25.0,185.0,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,"Speed Skating Women's 1,000 metres",NaN
8,5,Christine Jacoba Aaftink,F,27.0,185.0,82.0,Netherlands,NED,1994 Winter,1994,Winter,Lillehammer,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271094,135559,Pawe Jan Zygmunt,M,25.0,182.0,79.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Speed Skating,"Speed Skating Men's 1,500 metres",NaN
271095,135559,Pawe Jan Zygmunt,M,25.0,182.0,79.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Speed Skating,"Speed Skating Men's 5,000 metres",NaN
271096,135559,Pawe Jan Zygmunt,M,29.0,182.0,79.0,Poland,POL,2002 Winter,2002,Winter,Salt Lake City,Speed Skating,"Speed Skating Men's 5,000 metres",NaN
271097,135559,Pawe Jan Zygmunt,M,29.0,182.0,79.0,Poland,POL,2002 Winter,2002,Winter,Salt Lake City,Speed Skating,"Speed Skating Men's 10,000 metres",NaN


In [42]:
# dropna NaN's in medal column:
speed_skating = speed_skating.dropna(axis=0, subset='Medal')
speed_skating

# remove "Speed Skating" som är onödig:
speed_skating['Event'] = speed_skating['Event'].str.replace("Speed Skating ","")

C:\Users\danie\AppData\Local\Temp\ipykernel_9204\1088245756.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
speed_skating_gb_df = speed_skating.groupby(['Team','Medal','Event','Year'])['Medal'].count().reset_index(name='Medal Count')

# # sorterar ut  de 10 bästa länderna map antal guldmedaljer
# speed_sakting_sorted = speed_skating_gb_df[speed_skating_gb_df['Medal'] == 'Gold'].groupby('Team')['Medal Count'].sum().sort_values(ascending=False).reset_index()
# top_10_countries = speed_sakting_sorted.head(10)['Team'].tolist()
# speed_skating_df_top_10 = speed_skating_gb_df[speed_skating_gb_df['Team'].isin(top_10_countries)]
# speed_skating_df_top_10['Team'].sort_values()
# display(speed_skating_df_top_10.head())

# Mask out years with Speed Skating
years = speed_skating_gb_df['Year'].sort_values().unique().tolist()
display(years)

[1924,
 1928,
 1932,
 1936,
 1948,
 1952,
 1956,
 1960,
 1964,
 1968,
 1972,
 1976,
 1980,
 1984,
 1988,
 1992,
 1994,
 1998,
 2002,
 2006,
 2010,
 2014]

In [44]:
app = Dash(
    __name__,
    suppress_callback_exceptions=True,
    external_stylesheets=[dbc.themes.FLATLY],
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}]
)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Speed Skating Dashboard", className='text-center text-primary mx-3'),
            html.Br(),
            dcc.Dropdown(
                id="multi_dropdown",
                multi=True,
                searchable=False, 
                className='mb-2',
                placeholder='Select event',
                options=[event for event in speed_skating_gb_df['Event'].unique()],
                value=["Men's 1,500 metres"]
            ),
            html.Br(),
            html.P("Select year interval:"),
            dcc.RangeSlider(
                min=min(years),
                max=max(years),
                step=1,
                marks=None,
                id="RangeSlider",
                ),
            dcc.Graph(
                id='skate_graph',
                config={'staticPlot':True},
                figure={}
            ),
            html.Button("Download Image", id="download-img-btn", n_clicks=0),
            dcc.Download(id="img-downloader"),
            html.Br(),
            html.H1("Weightlifting: Age, Weight and Height Analysis over time."),
            html.P("Select class for analysis:"),
            dcc.RadioItems(
                options=[event for event in weightlifting['Event'].unique()],
                id="weightlifting-event",
                value="Women's Super-Heavyweight"
            ),
            html.P("Select analysis:"),
            dcc.RadioItems(
                ['Age','Weight','Height'],
                id="weightlifting-attr",
                value='Age'
            ),
            dcc.Graph(
                id='weightlifting-graph',
                config={'staticPlot':True},
                figure={}
            )
        ], xs=12, sm=11, md=10, lg=5)
    ], justify='center'),
], fluid=True)


@callback(
    Output("skate_graph", "figure"),
    Input("RangeSlider", "value"),
    Input("multi_dropdown", "value")
)

def update_multidropdown(year, events):
    if events is None: events = []
    if year is None: year = [1896, 1922]
    df = speed_skating_gb_df[(speed_skating_gb_df['Event'].isin(events)) & (speed_skating_gb_df['Year'] >= year[0]) & (speed_skating_gb_df['Year'] <= year[1])]
    return px.bar(df,
                x='Team',
                y='Medal Count',
                hover_name='Event',
                hover_data='Event',
                color='Medal',
                title=f'Speed Skating medals, between {year[0]} and {year[1]}:')

@callback(
    Output('img-downloader', 'data'),
    Input('download-img-btn', 'n_clicks' ),
    Input('skate-graph','figure')
)
def save_img(n_clicks, figure):
    if n_clicks:
        img_bytes = figure.to_image(format='png')
        return dcc.send_bytes(img_bytes, "../Images/speed_skate.png")
        
@callback(
    Output('weightlifting-graph','figure'),
    Input('weightlifting-event','value'),
    Input('weightlifting-attr', 'value')
)

def update_weightlifting(event, attr):
    event = [event] #måste göra om str till list
    if event is None: event = ["Women's Super-Heavyweight"]
    if attr is None: attr = []
    df = weightlifting[(weightlifting['Event'].isin(event))]
    return px.box(
        df,
        x='Year',
        y=attr,
        title=f'{attr} for {event[0]}.')

     
# app.run(host='0.0.0.0', port=8050,debug=True, jupyter_mode='external')
app.run(debug=True, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/
